# Chromosome assemblies NN

1. [https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/38p14_assembly_structure/Primary_Assembly/assembled_chromosomes/FASTA/](https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_assembly_structure/Primary_Assembly/assembled_chromosomes/FASTA/)

### Problems
1. Coordinates did not correspond to the nuccore sequencce on ncbi site
1. Test this version
1. https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_assembly_structure/Primary_Assembly/assembled_chromosomes/FASTA/

In [185]:
# GLOBALS
import sys
import json
import gzip
from hashlib import sha256
import os
from dotenv import load_dotenv

inputChr = 24

inputPath = 'C:\\Users\\creeperpandatrex\\Documents\\1000genomes\\DATA\\GCF000001405-38p14-assembly\\'

inputFile = 'chr'+str(inputChr)+'.fna.gz'
inputFile = 'chrY.fna.gz'
inputFile = 'chr'+str(inputChr)+'.fna.gz'
inputFile = 'chrX.fna.gz'

myLocal = inputPath + 'awk_ID_symbol_start_end_chr_'+str(inputChr)

# print(os.getenv('SEED'))


## Algorithm
1. read the gz file using fasta2one(gzip.open(inputpath+inputFile, 'rt)) syntax

In [186]:
# my is a string of chromosome 'NN' fasta files with 80 char per line to fasta with 2 lines ONLY
def fasta2one (myPath):
    with myPath as file:
        build = ''
        print(myPath)
        for i in file:
            build += i.strip()
        return build

my = fasta2one(gzip.open(inputPath + inputFile, 'rt')) # 6.27.2024 __14__ minutes

print(type(my)) # <class 'str'>


<_io.TextIOWrapper name='C:\\Users\\creeperpandatrex\\Documents\\1000genomes\\DATA\\GCF000001405-38p14-assembly\\chrX.fna.gz' encoding='utf-8'>
<class 'str'>


## Algorithm

1. zgrep -P 'chromosome\t22' GCF_000001405.40_GRCh38.p14_feature_table.txt > grep_minusP_chr_backslash_t_22 
1. PIPE wc -l => 4357
1. grep gene grep_minusP_chr_backslash_t_22 > grep_gene_minusP_chr22
1. PIPE wc -l  => 896
1. awk -F'\t' '{print$16"\t"$15"\t"$8"\t"$9}' grep_gene_minusP_chr22 > awk_ID_symbol_start_end_chr_22
1. read chrNUMB.fna.gz, output.strip(); output as string
1. read file:///C:/Users/creeperpandatrex/Documents/1000genomes/DATA/38.p14/awk_ID_symbol_start_end_chr_22, 
1. as list per element/line [id, symbol, start, end]
1. build a set to eliminate duplicates
1. with both files 
    1. read [id,symbol,start,end] from one file
    1. extract sequence using start, end+1, end-start, with counter equal to start
    1. hash the built sequence
    1. save idSymbol-map-to-idSymbolHash and idSymbolHash-map-to-buildSeqHash (maybe two or three)
    1. save as [{ 'idSymbHash': 'buildSeqHash'}, {}, {}, ....]
1. Upload sequence
1. done

In [187]:
# tmp list from set of grep_gene to remove duplicates
# myLocal = inputPath + 'awk_ID_symbol_start_end_chr_2'
def idSymbolBuildSet(myPath):
    print(myPath)
    myList = []
    obj = open(myPath, 'r')
    for i in obj:
        local = i.split()
        myList.append(local)
        local = []
    print(len(myList)) # 3362 total, after removal 2958
    mySet = set(tuple(i) for i in myList) # remove duplicates 
    set2list = list(mySet)
    return set2list

tmp = idSymbolBuildSet(myLocal)
print(len(tmp))

myRange = 10
counter = 0
for i in range(myRange):
    print(tmp[counter])
    counter += 1

# https://stackoverflow.com/questions/26783326/how-to-make-a-set-of-listslist
# >>> l = [[1, 2, 3], [2, 4, 5], [1, 2, 3], [2, 4, 5]]
# >>> set(tuple(i) for i in l)
# {(1, 2, 3), (2, 4, 5)}

C:\Users\creeperpandatrex\Documents\1000genomes\DATA\GCF000001405-38p14-assembly\awk_ID_symbol_start_end_chr_24
2512
2375
('107985659', 'LOC107985659', '52583989', '52589040')
('6845', 'VAMP7', '155881345', '155943769')
('100125392', 'TIPINP1', '53455216', '53457148')
('100302153', 'MIR1298', '114715233', '114715344')
('100873985', 'MED14OS', '40735690', '40738698')
('642546', 'HK2P1', '80569430', '80575038')
('100129818', 'TUBAP6', '115424045', '115425694')
('100873568', 'RNA5SP517', '146427456', '146427572')
('7317', 'UBA1', '47190847', '47215128')
('101927830', 'LOC101927830', '155347137', '155374174')


### Two files
1. chr22 as string my
1. idSymbol with start and end coordinates tmp is a list of 4 tuples
1. tmp = [ (id,symbol,start,end), (), (), (), ......]
1. hash all four, meaning each tuple

## Output
1. processing is the    return [myListOfDicMap, myListIdSymbSeqHash]
1. myListOfDicMap is [ {'fourTuple': '(id, symb, start, end)', 'fourTupleHash':'XXXXXXX'}, {}, {}, .....]
1. myListIdSymbSeqHash is [ {fourTupleHash : geneSeqHash}, {'XXXXX' : 'XXXXXXX' }, {'XX':'XX'}, .... ]
1. Process chromosome 22 3.2 seconds
1. Compress and Write to disk object: processing []
1. Pending sanity test with 5' 3' primers from each gene to confirm 

In [188]:
## Saving into a file


In [189]:
def processChrWithIdSymbolSanityCheck():
    myListOfDicMap = []
    myListIdSymbSeqHash = []
    sanityCheck = []
    primer = ''
    for i in range(len(tmp)):
        localArray = []
        mySeed = os.getenv('SEED')
        myDicMap = {}
        myDicTupleSeqHash = {}
        localTuple = tmp[i]
        localTupleHash = sha256((convertTuple(localTuple) + mySeed).encode()).hexdigest()
        myDicMap.update({'fourTuple':localTuple, 'fourTupleHash':localTupleHash})
        myListOfDicMap.append(myDicMap)
        seqHash = buildSeqHash(localTuple, mySeed)
        myDicTupleSeqHash.update( {localTupleHash: seqHash[0] })
        # print(type(seqHash[1]))
        seqSize = len(seqHash[1])
        print('>'+str(localTuple)+'\t'+str(seqSize)+'\n')
        primer += '>'+str(localTuple)+'\t'+str(seqSize)+'\n'
        first = ''
        last = ''
        counter = 0
        if len(seqHash[1]) > 40:
            for j in range(40):
                first += seqHash[1][counter]
                counter += 1
            localArray.append(first)
        counter =  -40
        if len(seqHash[1]) > 40:
            for k in range(40):
                last += (seqHash[1][counter])
                counter += 1
            localArray.append(last)
        # print(first+'..'+len(seqHash)+'..'+last)   #### PRINT
        # print(first+'....'+last)   #########  PRINT
        primer += first+'....'+last+'\n'
        sanityCheck.append(seqHash[1])
        myListIdSymbSeqHash.append(myDicTupleSeqHash)
    return [myListOfDicMap, myListIdSymbSeqHash, sanityCheck, primer]    
    
def print2file (myPath, result):
    out = open(myPath, 'w')
    out.write(result) # it must be a string

def buildSeqHash(tuple, mySeed):
    mySeq = ''
    id,symb,start,end = tuple
    start = int(start) 
    end = int(end)  
    counter = start + 80
    for i in range(end - start + 1):
        mySeq += my[counter]
        counter += 1
    mySeqHash = sha256(((mySeq+mySeed)).encode()).hexdigest()
    return [mySeqHash, mySeq] # both string


# print(buildSeq(processing[0]['fourTuple']))
        
def convertTuple(tup):
    str = ''.join(tup)
    return str

processing = processChrWithIdSymbolSanityCheck()
print(len(processing)) # 4; 2 minutes



# counter = 0
# for i in range(10):
#     print(processing[0][counter]) # labelmap list of dics json
#     print(processing[1][counter]) # labelseqhash list of dics json
#     print(processing[2][counter]) # gene sequence in array
#     print(processing[3][counter]) # primer string
#     counter += 1

# {'fourTuple': ('105373037', 'LOC105373037', '40555177', '40564466'), 'fourTupleHash': 'b1514454efb34d796e695624a103c9a5b4727f09df995728389633123747a653'}
# {'fourTuple': ('339685', 'LOC339685', '47345571', '47373541'), 'fourTupleHash': 'b5c159d961591a4e83f3726b4933d92890392c769b28c825c4bc0b2d05bca5d8'}


>('107985659', 'LOC107985659', '52583989', '52589040')	5052

>('6845', 'VAMP7', '155881345', '155943769')	62425

>('100125392', 'TIPINP1', '53455216', '53457148')	1933

>('100302153', 'MIR1298', '114715233', '114715344')	112

>('100873985', 'MED14OS', '40735690', '40738698')	3009

>('642546', 'HK2P1', '80569430', '80575038')	5609

>('100129818', 'TUBAP6', '115424045', '115425694')	1650

>('100873568', 'RNA5SP517', '146427456', '146427572')	117

>('7317', 'UBA1', '47190847', '47215128')	24282

>('101927830', 'LOC101927830', '155347137', '155374174')	27038

>('100420653', 'LOC100420653', '103258570', '103259318')	749

>('107985687', 'LOC107985687', '40101736', '40106486')	4751

>('106481876', 'RNU6-122P', '123874459', '123874565')	107

>('100128171', 'MTFR1P1', '66359581', '66361865')	2285

>('106480444', 'FCF1P9', '86481532', '86481813')	282

>('9104', 'RGN', '47078443', '47093313')	14871

>('106480889', 'RN7SKP183', '20452108', '20452431')	324

>('11043', 'MID2', '107825735', '10793163

## Output
1. processing is the     return [myListOfDicMap, myListIdSymbSeqHash, sanityCheck, primer]    
1. myListOfDicMap is [ {'fourTuple': '(id, symb, start, end)', 'fourTupleHash':'XXXXXXX'}, {}, {}, .....]
1. extension: `labelmap` list of dics of json
1. myListIdSymbSeqHash is [ {fourTupleHash : geneSeqHash}, {'XXXXX' : 'XXXXXXX' }, {'XX':'XX'}, .... ]
1. extension: `labelseqhash` list of dics of json
1. sanityCheck, extension: gene sequence in array
1. primer: text file two lines, first line a tuple(id, symbol, start, end), second line 5'3' primers of gene genomic seq
1. Process chromosome 21: ___ seconds
1. Compress and Write to disk object: processing []
1. Pending sanity test with 5' 3' primers from each gene to confirm 

In [190]:
# 
# print2file(inputPath + inputFile + ".labelmap", json.dumps(dict2hash(globalDictionary)[0]))
# print2file(inputPath + inputFile + ".labelseqhash", json.dumps(dict2hash(globalDictionary)[1]))
result = processing

def print2file (myPath, result):
    out = open(myPath, 'w')
    out.write(result) # it must be a string

def print2gz (myPath, result, addExt):
   with gzip.open(myPath + addExt, 'w') as f:
    f.write((json.dumps(result)).encode()) # no decode() since it is a str already

print(inputPath+inputFile)
print2gz(inputPath + inputFile, result[0], '.labelmap.gz')
print2gz(inputPath + inputFile, result[1], '.labelseqhash.gz')

C:\Users\creeperpandatrex\Documents\1000genomes\DATA\GCF000001405-38p14-assembly\chrX.fna.gz


In [191]:
### this is SANITY CHECK  
### processing[0] is .labelmap list of dics json
### processing[1] is  .labelseqhash list of dics json
### processing[2] is .fna.gene array of sequences from gene
### processing[3] is .primer string 2 lines 

print(len(processing[2])) # 2958

print(inputPath+inputFile)

print2file(inputPath+inputFile+'.primer', processing[3])
# print2file(inputPath+inputFile+'.fna.gene', json.dumps(processing[2]))

2375
C:\Users\creeperpandatrex\Documents\1000genomes\DATA\GCF000001405-38p14-assembly\chrX.fna.gz


In [192]:
# Sanity check
# {'fourTuple': ('105373037', 'LOC105373037', '40555177', '40564466'), 'fourTupleHash': 'b1514454efb34d796e695624a103c9a5b4727f09df995728389633123747a653'}
# {'fourTuple': ('339685', 'LOC339685', '47345571', '47373541'), 'fourTupleHash': 'b5c159d961591a4e83f3726b4933d92890392c769b28c825c4bc0b2d05bca5d8'}


## Checking genome sequences between primers
### this is SANITY CHECK  
1. processing[0] is .labelmap
1. processing[1] is  .labelseqhash
1. processing[2] is .fna.gene
1. processing[3] is .primer
1. sanity check with processChrWithIdSymbolSanityCheck(): primers good pending 
double check hashes
1. not sanity check with processChrWithIdSymbol(): hashesh wrong and not checked



### Pending
1. 6.23.2024
1. .primer is two lines: first line with > and tuple with id, symbol, start, end
1. .primer second line with gene 5'primer 3'end primer 
1. direction based in reference sequence
1. Checked 
```
 19 >('5902', 'RANBP1', '20116104', '20127355')     11252
     20 ACTGTCCATAGAGGGGTCACCACGTCGGCCACTCGTGTTA....CATCAAAAATAGTGAATAAAAAACACATTTGGAACCTGGG
  
```
And it looks good

In [193]:
print(len(processing[2])) # 2958

2375
